In [16]:
import numpy as np
import pandas as pd

import os
from scipy.integrate import trapezoid
import pathlib

In [ ]:
base_dir = os.path.join(pathlib.Path.home(), 'ecen5407', 'ecen5407_project2')

data_pt = os.path.join(base_dir, "data", "test_cases", "Trial_Full_System_90kW_3hr_Battery_with_Geothermal_Ramp_Limits","Trial_Full_System_90kW_3hr_Battery_with_Geothermal_Ramp_Limits_system_info.csv")
system_info = pd.read_csv(data_pt)

load_pt = os.path.join(base_dir, "data", "PySam_Outputs", "baseline_system_output.csv")
load = pd.read_csv(load_pt, index_col="Datetime", usecols=["Datetime", "Load (MW)"])
load.set_index(pd.DatetimeIndex(load.index), inplace=True)

In [ ]:
x = np.arange(0, 8760, 5/60)
y = np.ravel(load.values)
annual_load = trapezoid(y, x) # MWh
annual_load_kwh = annual_load*1000

In [49]:
annual_load

np.float64(407720.13880330476)

In [99]:
ilr = 1.33
m2_per_wdc = 0.2*0.001
0.29 + 0.04/ilr + 0.16 + 0.18 + m2_per_wdc*(27.07 + 92.11) + 0.044 + 0.02 + 0.03 + m2_per_wdc*2.5 + 0.002

0.7804111879699247

In [100]:
transmission_cost_per_mile = 6000000

geothermal_rating = 87000
geo_binary= 1

wind_capex = system_info['Wind Cost']
wind_op_costs = 30.85*system_info["Wind System Size"]
# wind_op_costs = system_info["Wind Operating Cost"]
wind_costs = {'Operating Costs': wind_op_costs, 'Capex': wind_capex}

geothermal_cost_per_kwac = 6153.66
geothermal_capex = 26.6*transmission_cost_per_mile*geo_binary + geothermal_cost_per_kwac*87*1000
geothermal_op_costs = 118.41*geothermal_rating*geo_binary
geo_costs = {'Operating Costs': geothermal_op_costs, 'Capex': geothermal_capex}

pv_capex = system_info["PV Cost"]
# pv_op_costs = system_info["PV Operating Cost"]
pv_op_costs = 48*system_info["PV System Size"] + 180000
battery_capex = system_info["Battery Cost"]

hybrid_capex = 676000000
# pv_capex + battery_capex
hybrid_op_costs = pv_op_costs
hybrid_costs = {'Operating Costs': hybrid_op_costs, 'Capex': hybrid_capex}

system_costs = {}
annual_op_costs =  wind_op_costs + geothermal_op_costs + hybrid_op_costs
system_costs['Annual Operating Costs'] = annual_op_costs

inflation_rate = 1.0025
annual_op_costs_by_year = [annual_op_costs*inflation_rate**i for i in range(1, 26)]
total_op_costs = sum(annual_op_costs_by_year)
avg_annual_op_costs = total_op_costs/25
system_costs['Average Annual Operating Costs'] = avg_annual_op_costs

upfront_cost = geothermal_capex + 783308096
real_interest_rate = 1 + (0.00614 - 0.0025)
real_upfront_cost = upfront_cost*real_interest_rate**25 
avg_annual_upfront_payment = real_upfront_cost/25
system_costs['System Capex'] = real_upfront_cost
system_costs['Average Annual Capex Payments'] = avg_annual_upfront_payment

avg_annual_total_payment = avg_annual_upfront_payment + avg_annual_op_costs
system_costs['Required Annual Average Revenue'] = avg_annual_total_payment

avg_energy_rate = avg_annual_total_payment/annual_load_kwh
system_costs['Energy Rate (kWh)'] = avg_energy_rate


In [101]:
system_costs

{'Annual Operating Costs': 0    2.340343e+07
 dtype: float64,
 'Average Annual Operating Costs': 0    2.417947e+07
 dtype: float64,
 'System Capex': 1618842963.157701,
 'Average Annual Capex Payments': 64753718.52630804,
 'Required Annual Average Revenue': 0    8.893319e+07
 dtype: float64,
 'Energy Rate (kWh)': 0    0.218123
 dtype: float64}

In [105]:
sum([hybrid_op_costs*inflation_rate**i for i in range(1, 26)])/25

0    1.159196e+07
Name: PV System Size, dtype: float64